In [1]:
from shapely.geometry import Point, Polygon
import fiona 
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%run ../python_files/data_cleaning

In [2]:
#import data
wards = gpd.read_file("../data/google_maps/gis/Wards.kml", driver='KML')
blocks_gis = gpd.read_file("../data/eviction_lab/gis/block-groups.geojson")
census_tracts_gis = gpd.read_file("../data/open_data/gis/Census_Tracts_in_2010.kml", driver='KML')
puds_gis = gpd.read_file("../data/open_data/gis/Planned_Unit_Development_PUDs.kml", driver='KML')
tifs_gis = gpd.read_file("../data/open_data/gis/Tax_Increment_Financing_TIF_areas.kml", driver='KML')
evict_data = pd.read_csv("../data/eviction_lab/csv/tracts.csv")
blocks = pd.read_csv("../data/eviction_lab/csv/block-groups.csv")
census_tracts = pd.read_csv('../data/open_data/csv/Census_Tracts_in_2010.csv')
puds = pd.read_csv('../data/open_data/csv/Planned_Unit_Development_PUDs.csv') 
tifs = pd.read_csv("../data/open_data/csv/Tax_Increment_Financing_TIF_areas.csv")
puds_xwalk = pd.read_csv('../data/grassroots_dc/csv/zoning_crosswalk.csv')
puds_gr = pd.read_csv('../data/grassroots_dc/csv/Grassroots PUD Data_njb - PUDs.csv')

In [3]:
#make tracts
evict16 = clean_eviction_data(evict_data)
tracts = merge_tracts(census_tracts_gis, census_tracts)
evict = merge_eviction_data(tracts, evict16)
evict = add_gis_ft1(wards, evict, 'ward')

In [5]:
#make puds
puds = merge_gis(puds_gis, puds)
puds_gr = clean_grassroots_pud_data(puds_gr)
puds = add_pud_ft(puds, puds_xwalk, puds_gr)
puds = add_gis_ft2(tracts, puds, 'GEOID')

AttributeError: 'DataFrame' object has no attribute 'geometry'

In [7]:
puds
# export_master(puds, 'linked_pud_tracts')

OBJECTID_PUD  \
0    64148          
1    64149          
2    64150          
3    64151          
4    64152          
5    64153          
6    64154          
7    64155          
8    64156          
9    64176          
10   64177          
11   64178          
12   64179          
13   64180          
14   64181          
15   64182          
16   64157          
17   64158          
18   64159          
19   64160          
20   64162          
21   64163          
22   64164          
23   64165          
24   64166          
25   64161          
26   64167          
27   64168          
28   64169          
29   64170          
..     ...          
256  64053          
257  64054          
258  64055          
259  64056          
260  64057          
261  64058          
262  64059          
263  64060          
264  64061          
265  64063          
266  64064          
267  64065          
268  64066          
269  64067          
270  64068          
271  64062          
272  64069          
273  64070          
274  64071          
275  64072          
276  64073          
277  64074          
278  64075          
279  64076          
280  64077          
281  64078          
282  64079          
283  64080          
284  64081          
285  64082          

                                                                 PUD_NAME  \
0    Matthews Memorial                                                      
1    Sheridan Redevelopment LLC & DCHA                                      
2    Alpine Associates                                                      
3    Parcel Seven Associates LLC                                            
4    Stanton-Eastbanc LLC                                                   
5    Salvation Army                                                         
6    Jemals Babes                                                           
7    Capper/Carrollsburg Venture                                            
8    GWU - Foggy Bottom Campus                                              
9    J River 1401 Pennsylvania Ave LLC                                      
10   District of Columbia Water and Sewer Authority                         
11   Level 2 Development                                                    
12   TBSC Owner I, LLC                                                      
13   Watkins Alley LLC                                                      
14   JBG/Boundary 1500 Hary Thomas LLC & JBG/Boundary Eckington Place LLC   
15   301 FL Manager LLC                                                     
16   St. Elizabeths Hospital                                                
17   USL WDC H Street LLC & H Street Self Storage LLC                       
18   Jemals Up Against the Wall                                             
19   Four Points LLC                                                        
20   Capitol Gateway Marketplace                                            
21   K Street Developers LLC                                                
22   Skyland Holdings, LLC                                                  
23   Forest City Washington                                                 
24   O Street Roadside LLC                                                  
25   GWU - Site 77A                                                         
26   1437-1451 Belmont Street N.W.                                          
27   2126 Wyoming Avenue NW                                                 
28   Abdo Development LLC                                                   
29   2300 Pennsylvania Avenue LLC                                           
..                            ...                                           
256  Valley Green/Skytower                                                  
257  Tenley Park LLC                                                        
258  Blair Road LP                                                          
259  Ox

In [6]:
#make tifs
tifs = merge_gis(tifs_gis, tifs)
tifs = add_gis_ft2(tracts, tifs, 'GEOID')

In [10]:
#make master files
master_puds = create_master(puds, evict)
master_puds = clean_master_puds(master_puds)
export_master(master_puds, 'master_puds_tracts2')
master_tifs = create_master(tifs, evict)
export_master(master_tifs, 'master_tifs_tracts')